In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import blimpy as bl 
import glob
import os
%matplotlib inline

# intended as a script-style notebook
# extracting all the necessary parameters, mainly from the filterbank headers, takes a long time
# and as such I typically run this in a screen session

In [28]:
print(np.array(mjds)[:10])

[60358. 60358. 60358. 60358. 60358. 60358. 60358. 60358. 60358. 60358.]


In [3]:
df0 = pd.read_csv('/datax/scratch/benjb/bl_nearby_stars/star_cadences_5.1_catalog.csv')

In [8]:
for i in range(len(df0.iloc[np.where(df0['Receiver'].values == 'Rcvr8_10')])):
    print(df0.iloc[np.where(df0['Receiver'].values == 'Rcvr8_10')]['.h5 path'].values[i])

/datag/pipeline/AGBT23B_999_43/blc40_blp00/blc40_guppi_60358_83848_GJ1002_0047.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc40_blp00/blc40_guppi_60358_84168_HIP792_0048.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc40_blp00/blc40_guppi_60358_84489_GJ1002_0049.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc40_blp00/blc40_guppi_60358_84814_HIP897_0050.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc40_blp00/blc40_guppi_60358_85138_GJ1002_0051.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc40_blp00/blc40_guppi_60358_85456_HIP904_0052.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc41_blp01/blc41_guppi_60358_83848_GJ1002_0047.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc41_blp01/blc41_guppi_60358_84168_HIP792_0048.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc41_blp01/blc41_guppi_60358_84489_GJ1002_0049.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc41_blp01/blc41_guppi_60358_84814_HIP897_0050.rawspec.0000.h5
/datag/pipeline/AGBT23B_999_43/blc41_blp01/blc41_g

In [5]:
h5_path = df0['.h5 path'].values[0]
fb = bl.Waterfall(h5_path, load_data=False)
print(fb.header)
fb.info()

{'DIMENSION_LABELS': array([b'time', b'feed_id', b'frequency'], dtype=object), 'az_start': 0.0, 'data_type': 1, 'fch1': 8001.464842353016, 'foff': -2.7939677238464355e-06, 'machine_id': 20, 'nbits': 32, 'nchans': 536870912, 'nifs': 1, 'source_name': 'GJ1002', 'src_dej': <Angle -7.5475 deg>, 'src_raj': <Angle 0.1119 hourangle>, 'telescope_id': 6, 'tsamp': 18.253611008, 'tstart': 57725.012824074074, 'za_start': 0.0}

--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :            8001.464842353016 MHz
            foff :      -2.7939677238464355e-06 MHz
      machine_id :                               20
           nbits :                               32
          nchans :                        536870912
            nifs :                                1
     source_name :                           GJ1002
         src_dej :                        

In [ ]:
dats = glob.glob('/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/*.dat')
print(len(dats))
udats = np.unique(dats)
print(len(udats))
lfvec = []
ldvec = []
lsvec = []
sfvec = []
sdvec = []
ssvec = []
cfvec = []
cdvec = []
csvec = []
xfvec = []
xdvec = []
xsvec = []

semester_counts = []

for i, dat in enumerate(udats):
    #print(dat)

    argwhere = np.where(df0['.dat path'].values == dat)[0]
    receiver = df0['Receiver'].values[argwhere][0]
    h5_path = df0['.h5 path'].values[argwhere][0]
    semester = h5_path.split('AGBT')[1][:3]
    cadence = df0['Cadence ID'].values[argwhere][0]

    if (cadence == 1549) | (cadence == 6762) | (cadence == 1281) | (cadence == 6363):
        continue

    df = pd.read_table(dat, sep='\s+', names=['Top_Hit_#','Drift_Rate','SNR','Uncorrected_Frequency','Corrected_Frequency',
                                                    'Index', 'freq_start', 'freq_end', 'SEFD', 'SEFD_freq', 'Coarse_Channel_Number', 
                                                    'Full_number_of_hits'], skiprows=9)
    if receiver == 'Rcvr1_2':
        tdf = df.iloc[
            np.where((df['Uncorrected_Frequency'].values > 1100) & (df['Uncorrected_Frequency'].values < 1200) |
                    (df['Uncorrected_Frequency'].values > 1340) & (df['Uncorrected_Frequency'].values < 1900))[0]
            ]
        f = tdf['freq_start'].values
        d = tdf['Drift_Rate'].values
        s = tdf['SNR'].values
        lfvec.append(f)
        ldvec.append(d)
        lsvec.append(s)
    elif receiver == 'Rcvr2_3':
        tdf = df.iloc[
            np.where((df['Uncorrected_Frequency'].values > 1800) & (df['Uncorrected_Frequency'].values < 2300) |
                    (df['Uncorrected_Frequency'].values > 2360) & (df['Uncorrected_Frequency'].values < 2700))[0]
            ]
        f = tdf['freq_start'].values
        d = tdf['Drift_Rate'].values
        s = tdf['SNR'].values
        sfvec.append(f)
        sdvec.append(d)
        ssvec.append(s)
    elif receiver == 'Rcvr4_6':
        tdf = df.iloc[
            np.where((df['Uncorrected_Frequency'].values > 4000) & (df['Uncorrected_Frequency'].values < 7800))[0]
            ]
        f = tdf['freq_start'].values
        d = tdf['Drift_Rate'].values
        s = tdf['SNR'].values
        cfvec.append(f)
        cdvec.append(d)
        csvec.append(s)
    elif receiver == 'Rcvr8_10':
        tdf = df.iloc[
            np.where((df['Uncorrected_Frequency'].values > 7800) & (df['Uncorrected_Frequency'].values < 11200))[0]
            ]
        f = tdf['freq_start'].values
        d = tdf['Drift_Rate'].values
        s = tdf['SNR'].values
        xfvec.append(f)
        xdvec.append(d)
        xsvec.append(s)
    if i%6 == 0:
        fb = bl.Waterfall(h5_path, load_data=False)
        ghz = fb.header['nchans']*2.7939677238464355e-06/1e3
    semester_counts.append([semester, receiver, len(f), ghz])

lfvec = np.concatenate(lfvec)
ldvec = np.concatenate(ldvec)
lsvec = np.concatenate(lsvec)
sfvec = np.concatenate(sfvec)
sdvec = np.concatenate(sdvec)
ssvec = np.concatenate(ssvec)
cfvec = np.concatenate(cfvec)
cdvec = np.concatenate(cdvec)
csvec = np.concatenate(csvec)
xfvec = np.concatenate(xfvec)
xdvec = np.concatenate(xdvec)
xsvec = np.concatenate(xsvec)

print(len(lfvec))
print(len(sfvec))
print(len(cfvec))
print(len(xfvec))

In [ ]:
ltvec = []
stvec = []
ctvec = []
xtvec = []
atvec = []

for s in np.unique(semester_arr[:,0]):
    print(s)
    li = np.where((semester_arr[:,1] == 'Rcvr1_2') & (semester_arr[:,0] == s))[0]
    si = np.where((semester_arr[:,1] == 'Rcvr2_3') & (semester_arr[:,0] == s))[0]
    ci = np.where((semester_arr[:,1] == 'Rcvr4_6') & (semester_arr[:,0] == s))[0]
    xi = np.where((semester_arr[:,1] == 'Rcvr8_10') & (semester_arr[:,0] == s))[0]
    semester_cut_L = semester_arr[li]
    semester_cut_S = semester_arr[si]
    semester_cut_C = semester_arr[ci]
    semester_cut_X = semester_arr[xi]
    lrfi = np.sum(semester_cut_L[:,2].astype('float')/semester_cut_L[:,3].astype('float'))/(len(li)/12)
    srfi = np.sum(semester_cut_S[:,2].astype('float')/semester_cut_S[:,3].astype('float'))/(len(si)/12)
    crfi = np.sum(semester_cut_C[:,2].astype('float')/semester_cut_C[:,3].astype('float'))/(len(ci)/12)
    xrfi = np.sum(semester_cut_X[:,2].astype('float')/semester_cut_X[:,3].astype('float'))/(len(xi)/12)
    arfi = (np.sum(semester_cut_L[:,2].astype('float'))/0.66+np.sum(semester_cut_S[:,2].astype('float'))/0.84+np.sum(semester_cut_C[:,2].astype('float'))/3.8+np.sum(semester_cut_X[:,2].astype('float'))/3.4)/5/(len(li)+len(si)+len(ci)+len(xi))
    ltvec.append(lrfi)
    stvec.append(srfi)
    ctvec.append(crfi)
    xtvec.append(xrfi)
    atvec.append(arfi)